In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pickle

DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

DATA_FILE = 'data/data.pkl'
with open(DATA_FILE, 'rb') as f:
    Cs = pickle.load(f)
    
DATA_FILE2 = 'data/labels.pkl'  
with open(DATA_FILE2, 'rb') as f:
    Ys = pickle.load(f)
    
with open(DATA_FILE3, 'rb') as f:
    pos_adjectives_list = pickle.load(f)
    
with open(DATA_FILE4, 'rb') as f:
    neu_adjectives_list = pickle.load(f)

with open(DATA_FILE5, 'rb') as f:
    neg_adjectives_list = pickle.load(f)

In [4]:
print(neu_adjectives_list)

['total', 'not', 'strawberry', 'filthy', 'less', 'unsatisfactory', 'first', 'new', 'so', 'regular', 'average', 'be', 'full', 'stodgy', 'different', 'slow', 'mostly', 'only', 'fast', 'unable', 'large', 'slowest', 'online', 'such', 'next', 'early', 'really', 'nt', 'last', 'entire', 'aerobic', 'young', 'the', 'its', 'mediocre', 'plain', 'loud', 'small', 'other', 'late', 'bearded', 'ice', 'second', 'busy', 'was', 'doable', 'iced', 'at', 'dry', 'old', 'decent', 'expensive', 'big', 'very', 'little', 'tasteless', 'few', 'unbearable', 'quick', 'a', 'newer', 'euro', 'national', 'crowded', 'above', 'vulgar', 'apolegetic', 'sweetlike', 'accurate', 'hot', 'bare', 'pretentious', 'raw', 'in', 'galbi', 'much', 'smaller', 'closed', 'grey', 'single', 'main', 'avondale', 'inexperienced', 'soooooo', 'classic', 'gotten', 'quite', 'slower', 'calgary', 'willing', 'local', 'same', 'chinese', 'waitress', 'just', 'available', 'oatmeal', 'an', 'eclectic', 'way', 'high', 'unique', 'bigger', 'goodsandwich', 'too'

In [5]:
import collections

collections.Counter(Ys[0])

Counter({1: 177, 2: 171, 3: 176, 4: 177, 5: 169})

In [6]:
print(f"Train Size: {len(Cs[0])}")
print(f"Dev Size:   {len(Cs[1])}")
print(f"Test Size:  {len(Cs[2])}")
print(f"Train Size: {len(Ys[0])}")
print(f"Dev Size:   {len(Ys[1])}")
print(f"Test Size:  {len(Ys[2])}")

Train Size: 870
Dev Size:   825
Test Size:  825
Train Size: 870
Dev Size:   825
Test Size:  826


In [8]:
from babble.utils import ExplanationIO

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO()
explanations = exp_io.read(FILE)

Read 595 explanations from data/my_explanations.tsv


In [ ]:
print(explanations[0].candidate)

In [9]:
from babble import Babbler
babbler = Babbler(Cs, Ys)

Grammar construction complete.


In [10]:
babbler.add_aliases({'positive': pos_adjectives_list})
babbler.add_aliases({'negative': neg_adjectives_list})
babbler.add_aliases({'neutral': neu_adjectives_list})

Grammar construction complete.
Grammar construction complete.
Grammar construction complete.


In [ ]:
babbler.apply(explanations, split=0)

/ |#                                                  | 0 Elapsed Time: 0:00:00

Building list of target candidate ids...
Collected 595 unique target candidate ids from 595 explanations.
Gathering desired candidates...
Could not find 595 target candidates with the following mention_ids (first 5):
-7137499708666345355
2054342241921923125
4248607722652347526
-6944025897051368090
1893822064894711740
Found 0/595 desired candidates
Linking explanations to candidates...
Linked 0/595 explanations


| |                                          #      | 594 Elapsed Time: 0:09:52


595 explanation(s) out of 595 were parseable.
4102 parse(s) generated from 595 explanation(s).
1988 parse(s) remain (2114 parse(s) removed by DuplicateSemanticsFilter).
Note: 1988 LFs did not have candidates and therefore could not be filtered.
1988 parse(s) remain (0 parse(s) removed by ConsistencyFilter).
Applying labeling functions to investigate labeling signature.
[===========================             ] 65%

In [ ]:
Ls = []
for split in [0,1,2]:
    L = babbler.get_label_matrix(split)
    Ls.append(L)

In [ ]:
parses = babbler.get_parses(translate=False)
print(len(parses))
for parse in parses:
    babbler.view_parse(parse)
    print(parse.function(candidate))

In [ ]:
from metal import LabelModel

label_aggregator = LabelModel(5)
label_aggregator.train(Ls[0], n_epochs=100, lr=0.01)
label_aggregator.score(Ls[1], Ys[1])

In [ ]:
Y_p = label_aggregator.predict(Ls[0])

In [ ]:
len_wrong_train = 0
for right, wrong in zip(Ys[0], Y_p):
    if right != wrong:
        len_wrong_train += 1
print(len_wrong_train)

In [ ]:
def most_frequent(line):
    filtered_list = list(filter(lambda a: a != 0, line))
    if len(filtered_list) > 0:
        occurence_count = Counter(filtered_list)
        return occurence_count.most_common(1)[0][0]
    else:
        return 0


def percentage(part, whole):
    return 100 * float(part)/float(whole)

In [ ]:
from collections import Counter

L_train = Ls[0].toarray()
L_test = Ls[2].toarray()

predicted_training_labels = []
predicted_test_labels = []

for line in L_train:
    predicted_training_labels.append(most_frequent(line))

print(predicted_training_labels)

for line in L_test:
    predicted_test_labels.append(most_frequent(line))
    
print(predicted_test_labels)

perturbations = []

len_wrong_train = 0
for right, wrong in zip(Ys[0], predicted_training_labels):
    if int(right) != wrong:
        len_wrong_train += 1
    perturbations.append(int(right) - wrong)

perturbations = [abs(perturbation) for perturbation in perturbations]

len_wrong_test = 0
for right, wrong in zip(Ys[2], predicted_test_labels):
    if right != wrong:
        len_wrong_test += 1

training_accuracy = percentage(len(Ys[0]) -len_wrong_train,len(Ys[0]))
test_accuracy = percentage(len(Ys[2]) - len_wrong_test, len(Ys[2]))

print("Number of wrong in training set: " + str(len_wrong_train))
print("Number of wrong in test set: " + str(len_wrong_test))
print("Training Accuracy: " + str(training_accuracy))
print("Test Accuracy " + str(test_accuracy))

In [ ]:
parses = babbler.get_parses(translate=False)

In [ ]:
parse = parses[110]
print(parse.explanation)
print(parse.semantics)

In [ ]:
element = Cs[0][250]
print(element)
print(element.text)
print(Ys[0][250])

In [ ]:
for parse in parses:
    label = parse.function(element)
    if label != 0:
        print(label)
        print(parse.explanation)

In [ ]:
for i,line in enumerate(L_train.T):
    if 4 in line:
        print(line)
        print(i)

In [ ]:
print(most_frequent(L_train[0]))

In [ ]:
wrong_indexes = []
i = 0
for right,wrong in zip(Ys[0],Y_p):
    if int(right) != wrong:
        wrong_indexes.append(i)
    i = i + 1

In [ ]:
print(len(wrong_indexes))

In [ ]:
with open("Ls.pkl", 'wb') as f:
    pickle.dump(Ls, f)
    
with open("Y_p.pkl", 'wb') as f:
    pickle.dump(Y_p, f)
    
with open("data/wrong_indexes.pkl", 'wb') as f:
    pickle.dump(wrong_indexes, f)